# Satisfiability Evaluator

This notebook evaluates TIG's satisfiability algorithms against standard academic benchmark instances of the 3-SAT problem, where each clause in the CNF formula has at most three literals.

**Useful Links:**
  * [Challenge Description](https://tig.foundation/challenges/satisfiability)
  * [Challenge Code](https://github.com/tig-foundation/tig-monorepo/blob/main/tig-challenges/src/satisfiability.rs)  

## 1. Setup Environment

### 1.1. Install Cargo

In [ ]:
import shutil
import os
if shutil.which("cargo") is None:
    print("cargo not found. Installing Rust and Cargo...")
    !curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y
    os.environ["PATH"] += f":{os.environ['HOME']}/.cargo/bin"
else:
    print("cargo is already installed.")

### 1.2. Clone tig-SOTA-metrics

In [ ]:
cwd = os.getcwd()
if cwd.endswith("tig-SOTA-metrics"):
    os.chdir("satisfiability_evaluator")
elif not cwd.endswith("satisfiability_evaluator"):
    !git clone https://github.com/tig-foundation/tig-SOTA-metrics
    os.chdir("tig-SOTA-metrics/satisfiability_evaluator")
else:
    print("already in the satisfiability_evaluator directory.")

### 1.3. Install Python Dependencies

In [ ]:
!pip install -r requirements.txt

### 1.4. Download Datasets

Currently, this evaluator has two placeholder datasets (SOTA datasets TBC):

- **2018_2024_3_SAT** SAT Competition (2018–2024): 3-CNF instances from the main track, used in SAT Competitions between 2018 and 2024. [Sourced from Global-Benchmark-Database](https://benchmark-database.de/)[^1]

- **SATLIB** Uniform Random 3-SAT: Randomly generated 3-SAT formulas from SATLIB Benchmark series. [Sourced from SATLIB - Benchmark Problems](https://www.cs.ubc.ca/~hoos/SATLIB/benchm.html)

[^1]: Markus Iser and Christoph Jabs. Global Benchmark Database. In 27th International Conference on Theory and Applications of Satisfiability Testing (SAT 2024). Leibniz International Proceedings in Informatics (LIPIcs), Volume 305, pp. 18:1-18:10, Schloss Dagstuhl – Leibniz-Zentrum für Informatik (2024) https://doi.org/10.4230/LIPIcs.SAT.2024.18

In [ ]:
datasets = ["2018_2024_3_SAT", "SATLIB"]
for ds in datasets:
    if not os.path.exists(f"data/{ds}"):
        !cd data && python3 download_{dataset}.py
    else:
        print(f"{ds} dataset already downloaded.")

## 2. Perform Evaluation

### 2.1. Fetch Top Earning Algorithms

Satisfiability was updated to Rust in round 25

In [ ]:
import requests

API_URL = "https://mainnet-api.tig.foundation"

print("Fetching block")
block = requests.get(f"{API_URL}/get-block").json()["block"]
curr_round = block["details"]["round"]
print(f"Current Round: {curr_round}")

algorithms = {
    x['id']: x
    for x in requests.get(f"{API_URL}/get-algorithms?block_id={block['id']}").json()["algorithms"]
}

print(f"Fetching Top Earning Algorithms for Satisfiability from Rounds 25 to {curr_round - 1}")
top_algos = []
for r in range(25, curr_round):
    data = requests.get(f"{API_URL}/get-round-emissions?round={r}").json()
    a_id, earnings = max(
        filter(
            lambda x: x[0].startswith("c001"),
            map(
                lambda x: (x[0], int(x[1]) / 1e18),
                data["algorithms"].items()
            )
        ),
        key=lambda x: x[1]
    )
    a_name = algorithms[a_id]['details']['name']
    print(f"Round: {r}, Algo: {a_name}, Round Earnings: {earnings:.2f} TIG")
    top_algos.append((r, a_name))

### 2.2. (Optional) Evaluate Local Algorithm

If you want to evaluate an algorithm that has not been submitted to TIG (e.g. you are preparing Advance Submission):

1. Add your algorithm code to `src/{ALGORITHM_NAME}.rs` and `src/{ALGORITHM_NAME}.cu`
2. Uncomment, edit, and run the below cell

In [ ]:
# top_algos.append((curr_round + 4, {ALGORITHM_NAME}))

### 2.3. Run Evaluations

Evaluation results are saved to `evaluations` folder as csv files

In [ ]:
unique_algos = set(x[1] for x in top_algos)
for a in unique_algos:
  for ds in datasets:
      !bash run.sh data/{ds} {a}

## 3. Plotting Results

Todo